<a href="https://colab.research.google.com/github/achanhon/coursdeeplearningcolab/blob/master/rl_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import gymnasium

GAMMA=0.99

class MemoryBuffer:
    def __init__(self):
        self.i = 0
        self.full = False

        L=50000
        self.s = torch.zeros(L,8)
        self.a = torch.zeros(L,4)
        self.r = torch.zeros(L)
        self.s_ = torch.zeros(L,8)
        self.f = torch.zeros(L)

    def push(self, s, a, r, s_, f):
        self.s[self.i] = s
        self.a[self.i][a] = 1
        self.r[self.i] = r
        self.s_[self.i] = s_
        self.f[self.i] = 1-f
        self.i += 1
        if self.i >= self.r.shape[0]:
            self.full = True
            self.i = 0

    def getBatch(self, B=64):
        if self.full:
            I = list((torch.rand(B) * self.r.shape[0]).long())
        else:
            I = list((torch.rand(B) * self.i).long())
        return (self.s[I], self.a[I], self.r[I], self.s_[I], self.f[I])

def trial(env,agent, T, memory):
    totalR = 0
    s, info = env.reset()
    for _ in range(1000):
        a = agent.sample(s,T)
        s_, r, terminated, truncated, info = env.step(a)

        memory.push(s,a,r,s_,terminated or truncated)
        totalR+=r
        if terminated or truncated:
            return totalR
        else:
            s = s_

def train(agent,T,memory,nbsteps):
    agent.cuda()
    optimizer = torch.optim.Adam(agent.parameters(), lr=0.0001, weight_decay=0.001)

    meanloss = torch.zeros(50).cuda()
    for step in range(nbsteps):
        B = memorybuffer.getBatch()
        S, A, R, S_,F = B
        S, A, R, S_,F = S.cuda(), A.cuda(), R.cuda(), S_.cuda(),F.cuda()

        Q = agent.Q(S)
        QA = (Q * A).sum(1)
        Q_ = agent.V(S_)
        loss = ((GAMMA * Q_ * F + R - QA)**2).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            meanloss[step % 50] = loss.clone()
            if step % 50 == 49:
                meanloss = meanloss.mean()
                print("\t", step, float(meanloss))
                meanloss = torch.zeros(50).cuda()


In [ ]:
env = gym.make("LunarLander-v3", render_mode="human")

T = 0.25

for i in range(10):
    for j in range(10):